In [2]:
%pwd

'd:\\sign_language_conversion\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'd:\\sign_language_conversion'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPickle:
    root_dir: Path
    data_raw: Path
    data_pickle: Path

In [6]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config['artifacts_root']])

    def get_data_config(self) -> DataPickle:
        config = self.config
        create_directories([config['data_pickle']['root_dir']])

        # Check if 'no_of_classes' exists in params
        if 'no_of_classes' not in self.params:
            raise KeyError("The key 'no_of_classes' is missing from params file.")

        data_pickle_config = DataPickle(
            root_dir=config['data_pickle']['root_dir'],
            data_raw=config['data_creation']['raw_dataset'],
            data_pickle=config['data_pickle']['pickle_dataset']
        )
        return data_pickle_config


In [8]:
import os
import cv2
import pickle
import mediapipe as mp

In [19]:
class DataFetcher:
    def __init__(self, config:DataPickle):
        self.config = config
    
    def datapick(self):
        try: 
            mp_hands = mp.solutions.hands
            hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

            data = []
            labels = []
            for dir_ in os.listdir(self.config.data_raw):
                for img_path in os.listdir(os.path.join(self.config.data_raw, dir_)):
                    data_aux = []
                    x_ = []
                    y_ = []

                    img = cv2.imread(os.path.join(self.config.data_raw, dir_, img_path))
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    results = hands.process(img_rgb)

                    if results.multi_hand_landmarks:
                        for hand_landmarks in results.multi_hand_landmarks:
                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y
                                x_.append(x)
                                y_.append(y)

                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y
                                data_aux.append(x - min(x_))
                                data_aux.append(y - min(y_))

                        data.append(data_aux)
                        labels.append(dir_)

            with open(self.config.data_pickle, 'wb') as f:
                pickle.dump({'data': data, 'labels': labels}, f)

            return 'data.pickle'

        except Exception as e:
            raise e

In [20]:
try:
    config = ConfigurationManager()
    data_pickle_config = config.get_data_config()
    data_creator = DataFetcher(config=data_pickle_config)
    data_creator.datapick()
except Exception as e:
    raise e

[2024-06-03 15:17:25,910: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-03 15:17:25,912: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-03 15:17:25,913: INFO: common: created directory at: artifacts]
[2024-06-03 15:17:25,914: INFO: common: created directory at: artifacts/data_pickle]


d:\sign_language_conversion\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
